# Baseline experiments

In [1]:

from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import MultiLabelBinarizer, normalize

from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from config.global_args import GlobalArgs

from src.preprocessing.get_preprocessed_data import get_preprocessed_data
from sklearn import preprocessing

import pickle

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import hamming_loss, accuracy_score, f1_score
import wandb
from sklearn.pipeline import Pipeline
import joblib 

In [6]:
# Get data: Preprocess or 

# Experiment config:
model_type = 'logistic_regression'

print(get_preprocessed_data(model_type))

#TODO change relative path
# with open('/Users/evaengel/comparison_NLP_classification_models/data/preprocessed_for_logistic_regression/preprocessed_data_en_all_levels.pkl', 'rb') as file:
#     dataset_dict = pickle.load(file)
# X_train, X_test, Y_train, Y_test = dataset_dict["X_train"], dataset_dict["X_test"], dataset_dict["Y_train"], dataset_dict["Y_test"]

Data already exists but will be overwritten.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evaengel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


305 classes were encoded by MultiLabelBinarizer.
preprocessed_data_en_all_levels.pkl at data/preprocessed_for_logistic_regression is overwritten.
None


In [8]:
model_type = 'bert'

df_train, df_test = get_preprocessed_data(model_type, overwrite_data = True)


Data already exists but will be overwritten.
305 classes were encoded by MultiLabelBinarizer.
preprocessed_data_en_all_levels.pkl at data/preprocessed_for_bert is overwritten.


ValueError: too many values to unpack (expected 2)

In [17]:
print(X_train.shape, Y_train.shape )

(29397, 1637432) (29397, 305)


# Fit Model with fixed params

In [ ]:
# Define logistic regression model
base_lr = LogisticRegression(solver='sag', random_state = GlobalArgs.random_seed, max_iter=1000)
# Define ClassifierChain for multi-label task 
cc_clf = ClassifierChain(base_lr)

In [ ]:
# Train logistic regression model with hyperparameter tuning
cc_clf.fit(X_train, Y_train)

# Hypertuning:

- RandomizedSearchCV
- GridSearch


In [20]:
# Define logistic regression model
base_lr = LogisticRegression(solver='sag', random_state = GlobalArgs.random_seed, max_iter=10)
# Define ClassifierChain for multi-label task 
clf = ClassifierChain(base_lr)

# Set up pipeline with vectorizer and classifier
pipeline = Pipeline([
    ('clf', clf)
])

# Set up hyperparameter grid for tuning
param_grid = {
    'clf__base_estimator__C': [0.1, 1, 10],
    'clf__base_estimator__penalty': ['l1', 'l2', 'elasticnet']
}

param_grid = {
    'clf__base_estimator__C': [0.1, 1],
    'clf__base_estimator__penalty': ['l1', 'l2']
}

# Set up grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='f1_micro')


In [21]:
grid_search.fit(X_train, Y_train)

/Users/evaengel/comparison_NLP_classification_models/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/evaengel/comparison_NLP_classification_models/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/evaengel/comparison_NLP_classification_models/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/evaengel/comparison_NLP_classification_models/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/evaengel/comparison_NLP_classification_models/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag

In [1]:
# Save the best model locally
best_model = grid_search.best_estimator_
joblib.dump(best_model, 'outputs/logistic_regression/best_model.joblib')

NameError: name 'grid_search' is not defined

In [ ]:
# Set up wandb
wandb.init(project='multi_label_classification')

In [ ]:
# Save the best model to wandb
wandb.run.summary['best_model'] = wandb.Artifact('best_model', type='model')
wandb.run.summary['best_model'].add_file('best_model.joblib')
wandb.log_artifact(wandb.run.summary['best_model'])

In [ ]:
predicitons = best_model.predict(X_test)

# Evaluation

In [ ]:
predicitons = cc_clf.predict(X_test)

In [ ]:
f1_average = metrics.f1_score(Y_test, predicitons, average= 'weighted')
print(f1_average)
f1 = metrics.f1_score(Y_test, predicitons)
print(f1)
lrap = metrics.label_ranking_average_precision_score(Y_test, predicitons)

print(lrap)

In [ ]:
hamming_loss_score = hamming_loss(Y_test, predicitons)
accuracy_score = accuracy_score(Y_test, predicitons)
f1_score_micro = f1_score(Y_test, predicitons, average='micro')
f1_score_macro = f1_score(Y_test, predicitons, average='macro')

# Log the evaluation metrics to wandb
wandb.log({'hamming_loss': hamming_loss_score,
           'accuracy_score': accuracy_score,
           'f1_score_micro': f1_score_micro,
           'f1_score_macro': f1_score_macro})

# Finish the wandb run
wandb.finish()

In [ ]:
# Log the evaluation metrics to wandb
wandb.log({'hamming_loss': hamming_loss_score,
           'accuracy_score': accuracy_score,
           'f1_score_micro': f1_score_micro,
           'f1_score_macro': f1_score_macro})

# Finish the wandb run
wandb.finish()